In [4]:
class Student:
    def __init__(self, name, batch, department, courses=None):
        self.name = name
        self.batch = batch
        self.department = department
        self.courses = list(set(courses)) if courses else []  # Ensure unique courses

    def add_courses(self, courses):
        self.courses = list(set(self.courses + courses))
    
    def remove_courses(self, courses):
        self.courses = list(set(self.courses).difference(courses))
    
    def __str__(self):
        course_names = [course.name for course in self.courses]
        return f"{self.name} ({self.batch}) - {self.department} - {', '.join(course_names)}"

In [5]:
class Course:
    def __init__(self, name, code, department, core_for=None):
        self.name = name
        self.code = code
        self.department = department
        self.core_for = core_for      # core_for = (batch, department)

    def __str__(self):
        return f"{self.name} ({self.code}) - {self.department}"

In [6]:
import pickle

class Management:
    def __init__(self, students=None, courses=None):
        self.students = list(set(students)) if students else []
        self.courses = list(set(courses)) if courses else []

    def add_students(self, students):
        self.students = list(set(self.students + students))

    def add_courses(self, courses):
        self.courses = list(set(self.courses + courses))

    def remove_students(self, students):
        self.students = list(set(self.students).difference(students))

    def remove_courses(self, courses):
        self.courses = list(set(self.courses).difference(courses))

    def get_students(self, department=None):
        if department:
            return [student for student in self.students if student.department == department]
        return self.students

    def get_courses(self, department=None):
        if department:
            return [course for course in self.courses if course.department == department]
        return self.courses

    def get_students_for_course(self, course_name):
        return [student for student in self.students if any(course.name == course_name for course in student.courses)]

    def get_courses_for_student(self, student):
        return [course for course in self.courses if course in student.courses]

    def get_core_courses(self, batch, department):
        if batch and department:
            return [course for course in self.courses if course.core_for == (batch, department)]
        elif batch and not department:
            return [course for course in self.courses if course.core_for[0] == batch]
        elif department and not batch:
            return [course for course in self.courses if course.core_for[1] == department]
        else:
            return [course for course in self.courses if course.core_for]
        
    def list_courses_with_student_count(self):
        course_student_count = {course: 0 for course in self.courses}
        
        for student in self.students:
            for course in student.courses:
                if course in course_student_count:
                    course_student_count[course] += 1

        sorted_courses = sorted(self.courses, key=lambda c: (c.department, c.core_for if c.core_for else ()))

        for course in sorted_courses:
            core_for_str = f" (Core for batch {course.core_for[0]})" if course.core_for else ""
            print(f"{course.name} ({course.code}) - {course.department}{core_for_str}: {course_student_count[course]} students")

    def save_to_file(self, filename):
        with open(filename, 'wb') as file:
            pickle.dump(self, file)

    @staticmethod
    def load_from_file(filename):
        with open(filename, 'rb') as file:
            return pickle.load(file)

    def __str__(self):
        return f"Management System with {len(self.students)} students and {len(self.courses)} courses"

In [39]:
# Example usage
course1 = Course("Data Structures", "CSE101", "CSE", (2020, "CSE"))
course2 = Course("Algorithms", "CSE102", "CSE", (2020, "CSE"))
course3 = Course("Circuits", "ECE101", "ECE", (2021, "ECE"))

student1 = Student("Alice", 2020, "CSE", [course1, course2])
student2 = Student("Bob", 2021, "ECE", [course3])

management = Management()
management.add_students([student1, student2])
management.add_courses([course1, course2, course3])

# Save to file
management.save_to_file('management_data.pkl')

# Load from file
loaded_management = Management.load_from_file('management_data.pkl')
print(loaded_management)

# Print details of loaded students and courses
for student in loaded_management.get_students():
    print(student)

for course in loaded_management.get_courses():
    print(course)

Management System with 2 students and 3 courses
Bob (2021) - ECE - Circuits
Alice (2020) - CSE - Data Structures, Algorithms
Circuits (ECE101) - ECE
Data Structures (CSE101) - CSE
Algorithms (CSE102) - CSE


In [40]:
import random

def generate_random_data():
    departments = ["SMS", "SPS", "SBS", "SCS", "SCoS", "SEPS", "SHSS"]
    batches = [2020, 2021, 2022, 2023, 2024]
    num_students = random.randint(1000, 1200)
    num_courses = random.randint(70, 80)

    # Generate courses
    courses = []
    for i in range(num_courses):
        department = random.choice(departments)
        batch = random.choice(batches) if department not in ["SEPS", "SHSS", "SCoS"] else 2024
        core_for = (batch, department) if department not in ["SEPS"] and random.random() < 0.5 else None
        course = Course(f"Course {i+1}", f"CODE{i+1}", department, core_for)
        courses.append(course)

    # Generate students
    students = []
    for i in range(num_students):
        name = f"Student {i+1}"
        batch = random.choice(batches)
        department = random.choice(departments)
        student_courses = [course for course in courses if course.core_for == (batch, department)]
        num_additional_courses = random.randint(0, max(0, 5 - len(student_courses)))
        additional_courses = random.sample([course for course in courses if course not in student_courses], num_additional_courses)
        student_courses.extend(additional_courses)
        student = Student(name, batch, department, student_courses)
        students.append(student)

    return students, courses

# Example usage
students, courses = generate_random_data()

management = Management()
management.add_students(students)
management.add_courses(courses)

# Save to file
management.save_to_file('management_data.pkl')

In [41]:
# Load from file
loaded_management = Management.load_from_file('management_data.pkl')
print(loaded_management)

# Print details of loaded students and courses
# for student in loaded_management.get_students():
#     print(student)

# for course in loaded_management.get_courses():
#     print(course)

# Print course with student count
loaded_management.list_courses_with_student_count()

Management System with 1052 students and 79 courses
Course 49 (CODE49) - SBS: 34 students
Course 50 (CODE50) - SBS: 36 students
Course 60 (CODE60) - SBS: 21 students
Course 79 (CODE79) - SBS: 22 students
Course 7 (CODE7) - SBS: 22 students
Course 67 (CODE67) - SBS: 27 students
Course 70 (CODE70) - SBS: 34 students
Course 77 (CODE77) - SBS (Core for batch 2020): 52 students
Course 43 (CODE43) - SBS (Core for batch 2020): 45 students
Course 24 (CODE24) - SBS (Core for batch 2021): 57 students
Course 75 (CODE75) - SBS (Core for batch 2021): 59 students
Course 35 (CODE35) - SBS (Core for batch 2021): 61 students
Course 41 (CODE41) - SBS (Core for batch 2022): 54 students
Course 47 (CODE47) - SBS (Core for batch 2023): 50 students
Course 48 (CODE48) - SBS (Core for batch 2024): 56 students
Course 15 (CODE15) - SBS (Core for batch 2024): 60 students
Course 29 (CODE29) - SBS (Core for batch 2024): 62 students
Course 62 (CODE62) - SCS: 29 students
Course 63 (CODE63) - SCS: 24 students
Course 6

In [40]:
class TimeTable:
    def __init__(self, management, num_slots=6):
        self.management = management
        self.courses = management.get_courses()
        self.students = management.get_students()
        self.num_slots = num_slots
        self.slots = [[] for _ in range(num_slots)]  # List of lists to store courses in each slot
        self.course_index = {course: idx for idx, course in enumerate(self.courses)}
        self.common_students_matrix = self.create_common_students_matrix()

    def create_common_students_matrix(self):
        num_courses = len(self.courses)
        matrix = [[0] * num_courses for _ in range(num_courses)]

        for student in self.students:
            for i in range(len(student.courses)):
                for j in range(i + 1, len(student.courses)):
                    course1 = student.courses[i]
                    course2 = student.courses[j]
                    idx1 = self.course_index[course1]
                    idx2 = self.course_index[course2]
                    matrix[idx1][idx2] += 1
                    matrix[idx2][idx1] += 1  # Since the matrix is symmetric

        return matrix

    def assign_slot(self, course, slot_number):
        if course in self.courses and 0 <= slot_number < len(self.slots):
            self.slots[slot_number].append(course)

    def check_clashes(self):
        clashes = 0
        for slot_courses in self.slots:
            for i in range(len(slot_courses)):
                for j in range(i + 1, len(slot_courses)):
                    course1 = slot_courses[i]
                    course2 = slot_courses[j]
                    idx1 = self.course_index[course1]
                    idx2 = self.course_index[course2]
                    clashes += self.common_students_matrix[idx1][idx2]
        return clashes
    
    def print_common_students_matrix(self):
        print("Common Students Matrix:")
        header = " " * 10 + " ".join(f"{course.code:^10}" for course in self.courses)
        print(header)
        for i, course1 in enumerate(self.courses):
            row = f"{course1.code:10}" + " ".join(f"{self.common_students_matrix[i][j]:^10}" for j in range(len(self.courses)))
            print(row)

    def greedy_slot_allocation(self):
        num_courses = len(self.courses)
        courses_per_slot = num_courses // self.num_slots
        extra_courses = num_courses % self.num_slots

        # Sort courses by the number of students enrolled in descending order
        sorted_courses = sorted(self.courses, key=lambda course: sum(self.common_students_matrix[self.course_index[course]]), reverse=True)

        # Assign courses to slots
        for course in sorted_courses:
            min_clashes = float('inf')
            best_slot = -1
            for slot_number in range(self.num_slots):
                if len(self.slots[slot_number]) < courses_per_slot + (1 if extra_courses > 0 else 0):
                    self.assign_slot(course, slot_number)
                    clashes = self.check_clashes()
                    if clashes < min_clashes:
                        min_clashes = clashes
                        best_slot = slot_number
                    self.slots[slot_number].remove(course)
            self.assign_slot(course, best_slot)
            if len(self.slots[best_slot]) == courses_per_slot + 1:
                extra_courses -= 1

    def __str__(self):
        result = "TimeTable:\n"
        for i, slot_courses in enumerate(self.slots):
            result += f"Slot {i+1}: " + ", ".join(course.name for course in slot_courses) + "\n"
        return result

In [42]:
loaded_management = Management.load_from_file('management_data.pkl')
print(loaded_management)

timetable = TimeTable(loaded_management)
# timetable.print_common_students_matrix()

# for i in range(len(timetable.courses)):
#     timetable.assign_slot(timetable.courses[i], i % 6)
timetable.greedy_slot_allocation()
print(timetable)

print(timetable.check_clashes())

Management System with 1052 students and 79 courses
TimeTable:
Slot 1: Course 42, Course 72, Course 14, Course 56, Course 2, Course 13, Course 41, Course 43, Course 38, Course 71, Course 45, Course 46, Course 23
Slot 2: Course 64, Course 73, Course 40, Course 3, Course 48, Course 50, Course 30, Course 62, Course 53, Course 10, Course 26, Course 66, Course 58, Course 79
Slot 3: Course 54, Course 19, Course 11, Course 35, Course 34, Course 77, Course 78, Course 69, Course 55, Course 33, Course 67, Course 61, Course 65
Slot 4: Course 27, Course 16, Course 18, Course 29, Course 31, Course 47, Course 39, Course 51, Course 74, Course 9, Course 1, Course 12, Course 63
Slot 5: Course 20, Course 21, Course 25, Course 24, Course 44, Course 32, Course 49, Course 76, Course 28, Course 4, Course 37, Course 60, Course 5
Slot 6: Course 17, Course 22, Course 36, Course 15, Course 75, Course 8, Course 68, Course 59, Course 70, Course 6, Course 57, Course 52, Course 7

509
